# Trabajo práctico N° 4

## Enunciado

Para los siguientes ejercicios vamos a usar el Wisconsin Breast Cancer dataset, construído a partir de imágenes digitalizadas. Las mismas describen la información obtenida de cada imagen de célula mamaria biopsiada, especificando si el diagnóstico sobre la misma es que es benigna o maligna.

Se pide:

1. Declarar una variable random_state igual al número de alumno en la hoja de cálculo "Entregas TPs" en el Google Drive de la materia.
2. Implementar un clasificador kNN.
3. Aplicarle z-score standarization a los datos. Dividir en conjunto de train y test y entrenar el clasificador con el sub-conjunto de datos de entrenamiento, mostrando el correspondiente error de test. Utilizar el correspondiente random_state.
4. Considere el caso donde un incorrecto diagnóstico de cáncer cuando la célula es benigna tiene un costo de 5, mientras que una errónea omisión de una célula maligna tiene un costo de 20. Modificar las salidas del clasificador para que el mismo realice la mayor reducción posible del costo por errores de clasificación, e imprimir dichas salidas. Ayuda: una forma es utilizar el método predicted_proba() para obtener las probabilidades.
5. Considerando el punto anterior, ampliar el código para que seleccione automáticamente o mediante iteraciones la cantidad de vecinos que minimiza el costo. Especificar cuál es el costo mínimo obtenido tras dicha minimización.

Fecha de entrega: 17/05/2017.
Nota: la resolución de los ejercicios es **individual**; en el caso de que dos ejercicios enviados contengan un código igual o muy similar (sin considerar los comentarios), se los considerará a ambos como desaprobados. La reutilización del código del notebook está permitida (por ejemplo para confeccionar gráficos).

## Desarrollo

#### 1. Se declara la variable random_state con el numero de alumno correspondiente (3) y se instancia el dataset breast_cancer

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
semilla = np.random.RandomState(3)
data = load_breast_cancer()
x = data.data
y = data.target

#### 2. Se Implementa un clasificador kNN

In [2]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(weights='distance', n_neighbors=10)

#### 3. Se aplica z-score standarization a los datos

In [3]:
%%capture
from scipy import stats
stats.zscore(x)
stats.zscore(y)

#### Se divide en conjunto de train y test y entrenar el clasificador con el sub-conjunto de datos de entrenamiento. Luego se muestra el error de test.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=semilla, test_size=0.2)
clf.fit(X_train, y_train)
print('Porcentaje de aciertos en la clasificación: ', clf.score(X_test, y_test))

Porcentaje de aciertos en la clasificación:  0.929824561404


#### 4. Se establecen los costos de lfn y lfp

In [5]:
lfn = 20
lfp = 5

#### Se define una funcion que modifica las salidas del clasificador teniendo en cuenta los costos definidos anteriormente

In [6]:
def predecir(X_test, lfn, lfp):
    c = lfn / lfp
    p = c / (1 + c)
    y_hat_proba = clf.predict_proba(X_test)
    for i_pred, pred in enumerate(y_hat_proba):
        for i_item, item in enumerate(pred):
            if item > p:
                # Si es mayor a la probabilidad le asignamos 1
                y_hat_proba[i_pred, i_item] = 1
            elif item < p:
                # Si es menor a la probabilidad le asignamos 0
                y_hat_proba[i_pred, i_item] = 0

    return y_hat_proba

#### Se predice y se muestra la salida

In [7]:
print(predecir(X_test, lfn, lfp))

[[ 0.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]

#### 5. Se busca el numero de vecinos $k$ que de el mayor porcentaje de aciertos 

In [8]:
k_optimo = 0
score_optimo = 0
for k in range(1, len(X_train)):
    clf = neighbors.KNeighborsClassifier(weights='distance', n_neighbors=k)
    clf.fit(X_train, y_train)
    if clf.score(X_test, y_test) > score_optimo:
        score_optimo = clf.score(X_test, y_test)
        k_optimo = k

print('El k optimo es: ', k_optimo, '. Da un porcentaje de aciertos de: ', score_optimo)

El k optimo es:  12 . Da un porcentaje de aciertos de:  0.938596491228
